# Statistical analysis of the longitudinal velocity profile of a turbulent jet

In this Jupyter Notebook, we shall analyze hot wire measurements of the velocity of a turbulent jet made in a wind tunnel at ENS de Lyon, by Christophe Baudet and Antoine Naert. A Jupyter notebook can contain the data analysis code (here written in Python), the figures and comments (using the [Markdown](https://daringfireball.net/projects/markdown/syntax) syntax). It can be version-controlled using `git`.

First, we import the modules we will need:
- `numpy` defines the array data structure and many tools to work with arrays. We shall use in particular the `numpy.statistics` submodule.
- `matplotlib` is the core module for making plots with Python. The best way to use `matplotlib` is to use the object-oriented API; for our need the central object is the [`matplotlib.pyplot.axes` object](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.axes.html#matplotlib.pyplot.axes). There is also an imperative style API which should look familiar to Matlab users.
- `scipy` (optional) contains more advanced tools for scientific computing. We are only importing the submodule dedicated to signal processing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.signal

You can find help with Python or the different libraries here:
- https://docs.python.org/3/
- https://docs.scipy.org/doc/numpy/reference/index.html
- https://matplotlib.org/api/pyplot_summary.html
- https://docs.scipy.org/doc/scipy/reference/signal.html

## Preamble: reading the data

Let us first get the data; since it is not too big we can afford keeping everything in memory.
We use a single file (using the HDF5 file format) containing 100 independent measurements of velocity $u(t)$ (in $m.s^{-1}$) at 2^20 instants, with a frequency of 23 kHz. The file also contains the sampling times and useful metadata, such as the sampling frequency and the viscosity of air. We read all this information from the file and store it in several variables in RAM.

In [ ]:
import h5py
with h5py.File('datajet.h5') as datafile:
    time = datafile['time'][:]
    data = datafile['velocity'][:]
    freq = datafile.attrs['sampl_freq']
    nu_air = datafile.attrs['viscosity']

Our data array is a matrix containing all the independent measurements. It takes roughly 400MB in RAM.

In [ ]:
data.shape

In [ ]:
4.*data.size/1024/1024

In [ ]:
data.nbytes/1024/1024

**Q1** Plot a few realizations of the signal to get familiar with the data and with `matplotlib`.

## I. One-point statistics

In this section we study one-point statistics (i.e. global quantities) of the signal.
We first estimate the mean velocity. Immediately after that, we centralize the data by subtracting the mean in each time series.

**Q2** Compute the mean velocity $U$, the standard deviation $\sigma$ and the turbulence intensity $I$. Discuss the validity of the Taylor hypothesis.

From now on we work with the centered signal $u(x) \to u(x)-\langle u \rangle$.
We also define the spatial resolution `dx`.

In [ ]:
data = data - np.mean(data, axis=-1, keepdims=True)
dx = U/freq

**Q3** Compute the probability density function for velocity. Compare it to a Gaussian density.

**Q4** Compute the mean energy dissipation rate $\varepsilon$, then the Taylor scale $\lambda$, the Taylor-scale Reynolds number $R_\lambda$, and the Kolmogorov scale $\eta$.

## II. Two-point statistics

Let us now move to two-point statistics: energy spectrum and, equivalently, autocorrelation function.
First, let us see how to efficiently evaluate the autocorrelation function.

### Second-order structure function

First a reminder of the definition of the longitudinal autocorrelation function:
$$ f(r)=\langle u(x)u(x+r)\rangle.$$

**Q5** Using the basic statistics functions in `numpy` (`np.mean`, `np.var` and `np.correlate`), define a function to evaluate the autocorrelation function of a signal directly from the definition, using the following prototype (write the code in the function and replace `NotImplemented` with the result):

In [ ]:
def autocorrelation_naive(x):
    """
    Compute autocorrelation function directly in real space
    http://en.wikipedia.org/wiki/Autocorrelation#Estimation
    """
    return NotImplemented

Check that it is slow (using only one velocity measurement). Then remember that the autocorrelation function can be computed from the power spectrum, based on the [Wiener-Khinchin theorem](https://en.wikipedia.org/wiki/Wiener%E2%80%93Khinchin_theorem), and define an alternative function to compute the autocorrelation function of a signal using the Discrete Fourier Transform (`np.fft.rfft`):

In [ ]:
def autocorrelation_spectral(x):
    """
    Compute autocorrelation function using FFTs
    Be careful with the normalization.
    """
    return NotImplemented

Plot the estimate of the autocorrelation function for one velocity measurement using the two methods (to gain time, you may do this test using only 1/10 of the time series for the naive method). You can also compare it to the estimate given by the function `scipy.signal.correlate` provided by the `scipy` library.
Then compare the performance of the three methods using the iPython magic [`%timeit`](https://ipython.readthedocs.io/en/stable/interactive/magics.html?highlight=timeit#magic-timeit).

Now estimate the autocorrelation function using all the available data. The following code should be compatible with your `autocorrelation_spectral` function (if not, fix it):

In [ ]:
autocorrel = np.mean(autocorrelation_spectral(data), axis=0)

You may compare this estimate with the autocorrelation function from individual measurements to check statistical convergence if you wish.

**Q6** Estimate the integral scale $L_0$.

Remember that velocity increments are defined by $\delta u(\ell) = u(x+\ell)-u(x)$, and structure functions by $S_n(\ell)=\langle \delta u(\ell)^n\rangle$.

**Q7** By relating it to the longitudinal velocity autocorrelation function, plot the second-order structure function $S_2$ as a function of scale. Can you identify a range of scales for which it behaves as a power law with exponent $2/3$? with exponent $2$? Annotate the graph by positioning the characteristic length scales $\eta$, $\lambda$ and $L_0$. Check that the asympotic value of $S_2(\ell)$ is $2 U_{\text{rms}}^2$.
To draw this figure, you may need the functions `matplotlib.pyplot.axhline` and `matplotlib.pyplot.axvline`.

### Energy spectrum

**Q8** Compute the energy spectrum $E(k)$ and plot it.

Again, check that the energy spectrum exhibits a power-law scaling. What is the exponent?
Identify the wave numbers associated to the scales $\eta, \lambda$ and $L_0$.

Compare the scaling ranges of the energy spectrum and the second-order structure function.

Tip: The Nyquist frequency is $1/(2dx)$, i.e. the maximum wave number is $\pi/dx$.
The function `np.fft.rfftfreq` returns sample frequencies for the Discrete Fourier Transform, we just need to multiply by $2\pi$ to get wave numbers.

**Q9** Estimate the Kolmogorov constant $C_K$.

### Third-order structure function: the 4/5 law

Unlike the second-order, there is no trick to compute efficiently the higher-order structure functions $S_n(\ell)$ ($n>2$). Let us first check that the direct computation using increments coincide with the above results for $S_2(\ell)$.

**Q10** Using 100 logarithmically spaced increments across the whole range of scales, compute $S_2(\ell)$ without using the velocity autocorrelation function and compare it with the above result.

You might need to use the `np.logspace` function, and to define the following function:

In [ ]:
def increment_scale(Lmin, Lmax, nincr):
    """
    Return the indices corresponding to nincr logarthmically spaced increments 
    between scales Lmin and Lmax.
    """
    return NotImplemented

Now let us use the same approach for the higher-order structure functions:

**Q11** Compute the third order structure function $S_3$ and plot it (you might reuse the function `increment_scale`). Is it in agreement with the $4/5$-law?

Reminder: the $4/5$-law states that
$$ \lim_{\ell \to 0} \lim_{\nu \to 0} \lim_{t \to +\infty} \frac{S_3(\ell)}{\ell} = -\frac{4}{5} \varepsilon.$$

### Intermittency

**Q12** Using the same method as for $S_3$, compute and plot the higher-order structure functions $S_n(\ell)$ ($n>3$). Do they exhibit a power-law scaling?

**Q13** Now compute and plot the probability density function for the velocity increments $\delta u (\ell)$ at various scales $\ell$. You should actually normalize the velocity increments by using the variable $\delta u(\ell)/\sqrt{S_2(\ell)}$. Are the velocity increments Gaussian?

**Q14** Plot the scaling exponents $\zeta(n)$ such that $S_n(\ell) \sim \ell^{\zeta(n)}$ in the inertial range for $n \leq 6$. Are the velocity increments self-similar?

Comment on the statistical convergence of the high-order moments.